In [14]:
import pandas as pd

In [2]:
df_groundtruth = pd.read_csv('ground_truth.csv')

In [3]:
def cleanup(row):
    oid = []
    for x in row[1][1:-1].split(sep=','):
        try:
            oid.append(int(x.replace(' ','')))
        except:
            pass
    if oid == []:
        return -1
    return oid

In [4]:
df_groundtruth['phenotype']=df_groundtruth.apply(lambda row: [x.replace("'",'').replace(' ','') for x in row[0][1:-1].split(sep=',')],axis=1)
df_groundtruth['oid'] = df_groundtruth.apply(cleanup,axis=1)
df_groundtruth = df_groundtruth[df_groundtruth['oid'] != -1]


In [5]:
df_groundtruth.reset_index(drop=True,inplace=True)
df_groundtruth

,phenotype,oid,gene
0,"[HP:0010301, HP:0000252, HP:0001263]",[353284],['EP300']
1,"[HP:0001263, HP:0002650, HP:0001382, HP:000207...",[1465],['SMARCB1']
2,"[HP:0001263, HP:0004322, HP:0100255]",[319671],['LARP7']
3,[HP:0007105],"[85201, 3047]",['KAT6B']
4,"[HP:0009372, HP:0001193, HP:0007598, HP:0001762]",[93384],['GDF5']
...,...,...,...
208,"[HP:0005616, HP:0000646, HP:0001263, HP:000199...","[79443, 79444]",['GNAS']
209,"[HP:0001263, HP:0001999, HP:0001510, HP:000138...",[3163],['PIK3R1']
210,"[HP:0000062, HP:0001328]","[242, 251510]",['NR5A1']
211,"[HP:0000365, HP:0000365, HP:0007703, HP:000066...","[53540, 791]",['NR2E3']


In [21]:
%%time
df_dataset = pd.read_parquet('ic_normalised_final.parquet')
oid_dataset = df_dataset.Orpha_ID.unique()
oid_dataset

CPU times: total: 19.2 s
Wall time: 23.2 s


array([   58,    61,   585, ..., 99925,  2843, 79492], dtype=int64)

In [22]:
def cleanup_1(row):
    oid = []
    for x in row[1][1:-1].split(sep=','):
        try:
            oid.append(int(x.replace("'",'').replace(' ','')))
        except:
            pass
    if oid == []:
        return -1
    return oid

In [23]:
#-2:test case not present in the ground truth
#1:correct
#0:incorrect
#-1:groundtruth not present in our dataset

In [52]:
df_results = pd.read_csv('dict_oid_res_test.csv')
df_results

,phenotype,oid
0,"['HP:0010301', 'HP:0000252', 'HP:0001263']",[353284]
1,"['HP:0001263', 'HP:0002650', 'HP:0001382', 'HP...",[1465]
2,"['HP:0001263', 'HP:0004322', 'HP:0100255']",[319671]
3,['HP:0007105'],"[85201, 3047]"
4,"['HP:0009372', 'HP:0001193', 'HP:0007598', 'HP...",[93384]
...,...,...
223,"['HP:0005616', 'HP:0000646', 'HP:0001263', 'HP...","[79443, 79444]"
224,"['HP:0001263', 'HP:0001999', 'HP:0001510', 'HP...",[3163]
225,"['HP:0000062', 'HP:0001328']","[242, 251510]"
226,"['HP:0000365', 'HP:0000365', 'HP:0007703', 'HP...","[53540, 791]"


In [53]:
df_results['phenotype']=df_results.apply(lambda row: [x.replace("'",'').replace(' ','') for x in row[0][1:-1].split(sep=',')],axis=1)
df_results['oid'] = df_results.apply(cleanup,axis=1)
df_results = df_results[df_results['oid'] != -1]
df_results.reset_index(drop=True,inplace=True)
df_results

,phenotype,oid
0,"[HP:0010301, HP:0000252, HP:0001263]",[353284]
1,"[HP:0001263, HP:0002650, HP:0001382, HP:000207...",[1465]
2,"[HP:0001263, HP:0004322, HP:0100255]",[319671]
3,[HP:0007105],"[85201, 3047]"
4,"[HP:0009372, HP:0001193, HP:0007598, HP:0001762]",[93384]
...,...,...
207,"[HP:0005616, HP:0000646, HP:0001263, HP:000199...","[79443, 79444]"
208,"[HP:0001263, HP:0001999, HP:0001510, HP:000138...",[3163]
209,"[HP:0000062, HP:0001328]","[242, 251510]"
210,"[HP:0000365, HP:0000365, HP:0007703, HP:000066...","[53540, 791]"


In [54]:
def score_accuracy(row):
    count = 0
    for x in range(df_groundtruth.shape[0]):
        if row[0] == df_groundtruth.at[x,'phenotype']:
            correct_disease = df_groundtruth.at[x,'oid']
            for y in correct_disease:
                if y in oid_dataset:
                    if y in row[1]:
                        count += 1
                else:
                    return -1
            if count == len(correct_disease):
                return 1
            else :
                return 0
    return -2

In [55]:
%%time
df_results['score'] = df_results.apply(score_accuracy,axis=1)

CPU times: total: 93.8 ms
Wall time: 170 ms


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [56]:
df_results.score.value_counts()

 1    157
-1     55
Name: score, dtype: int64

In [57]:
print(len(oid_dataset))

4240


In [36]:
df_results[df_results['score']== 1]

,phenotype,oid,score
0,"[HP:0010301, HP:0000252, HP:0001263]","[353277, 353284, 261552, 567, 904]",1
14,"[HP:0000558, HP:0000668, HP:0001999]","[3163, 904, 567, 261552, 3455]",1
83,"[HP:0100568, HP:0002668]","[29072, 892, 276621, 94080, 904]",1
98,"[HP:0000787, HP:0012444, HP:0001263, HP:000082...","[356961, 904, 2152, 1606, 261552]",1
195,"[HP:0012115, HP:0001399]","[1606, 567, 84, 280, 646]",1
206,"[HP:0002910, HP:0011157, HP:0001943, HP:000312...","[26791, 506, 1304, 100996, 254864]",1
221,[HP:0001067],"[97685, 363700, 904, 636, 363965]",1


In [13]:
df_results[df_results['score']== 1]

,phenotype,oid,score
0,"[HP:0010301, HP:0000252, HP:0001263]","[904, 567, 363965, 363958, 353284, 353277, 261...",1
9,"[HP:0001519, HP:0011342, HP:0002194]","[904, 84, 567, 363965, 363958, 636, 1606, 821,...",1
78,"[HP:0000787, HP:0012444, HP:0001263, HP:000082...","[904, 261552, 2152, 1606, 261537, 79324, 636, ...",1
157,"[HP:0012115, HP:0001399]","[1606, 567, 84, 280, 646, 356961, 79321, 2169,...",1
165,"[HP:0002910, HP:0011157, HP:0001943, HP:000312...","[506, 26791, 100996, 1304, 228123, 254864, 432...",1


In [182]:
df_results[df_results['score']== 1]

,phenotype,oid,score
0,"[HP:0010301, HP:0000252, HP:0001263]","[904, 261552, 353284, 353277, 2152, 280, 26153...",1
12,"[HP:0000558, HP:0000668, HP:0001999]","[3163, 904, 567, 261552, 3455, 353284, 353277,...",1
78,"[HP:0000787, HP:0012444, HP:0001263, HP:000082...","[904, 261552, 2152, 1606, 261537, 79324, 636, ...",1
165,"[HP:0002910, HP:0011157, HP:0001943, HP:000312...","[26791, 506, 1304, 100996, 254864, 228123, 206...",1
178,[HP:0001067],"[97685, 363700, 904, 636, 363965, 363958, 84, ...",1


In [147]:
df_groundtruth.loc[100]

phenotype    [HP:0000365, HP:0001263, HP:0000365, HP:001246...
oid                                                   [442835]
Name: 100, dtype: object

In [149]:
df_groundtruth

,phenotype,oid
0,"[HP:0010301, HP:0000252, HP:0001263]",[353284]
1,"[HP:0001263, HP:0002650, HP:0001382, HP:000207...",[1465]
2,"[HP:0001263, HP:0004322, HP:0100255]",[319671]
3,[HP:0007105],"[85201, 3047]"
4,"[HP:0009372, HP:0001193, HP:0007598, HP:0001762]",[93384]
...,...,...
208,"[HP:0005616, HP:0000646, HP:0001263, HP:000199...","[79443, 79444]"
209,"[HP:0001263, HP:0001999, HP:0001510, HP:000138...",[3163]
210,"[HP:0000062, HP:0001328]","[242, 251510]"
211,"[HP:0000365, HP:0000365, HP:0007703, HP:000066...","[53540, 791]"
